In [7]:
import datetime
from ib_insync import *
import pandas as pd

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [8]:
ib = connect_to_IB(clientid=3)

Peer closed connection. clientId 3 already in use?
API connection failed: TimeoutError()


TimeoutError: 

# Develop a method to retrieve the current portfolio as dataframe and store it in arcticdb

In [3]:
pm = PortfolioManager(ib)

In [9]:
pm.fetch_portfolio()

/Users/johanneslange/Desktop/Pythonprojekte/IB-Multi-Strategy-ATS/broker/portfolio.py:134: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([portfolio_df, cash_df], ignore_index=True)


,symbol,contractType,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,conId,lastTradeDate,account
0,EWT,STK,200.000,44.971699,8994.340,44.256828,142.97,0.0,253190576,,U7706434
1,IAU,STK,300.000,38.189701,11456.910,38.454255,-79.37,0.0,474829650,,U7706434
2,IAU,OPT,-2.000,0.166567,-33.310,23.350200,13.39,0.0,671974250,20240216,U7706434
3,INDA,STK,100.000,49.170799,4917.080,49.712499,-54.17,0.0,101484826,,U7706434
4,IUSQ,STK,100.000,70.419998,7042.000,65.602785,481.72,0.0,239363241,,U7706434
5,IWM,STK,100.000,195.679993,19568.000,199.616920,-393.69,0.0,9579970,,U7706434
6,IWM,OPT,-1.000,2.743192,-274.320,183.368900,-90.95,0.0,637114226,20240216,U7706434
7,PRX,STK,250.000,28.260000,7065.000,26.561554,424.61,0.0,382625193,,U7706434
8,SOXX,OPT,-1.000,0.852627,-85.260,699.944800,614.68,0.0,653437229,20240216,U7706434
9,TLT,STK,100.000,93.870003,9387.000,99.051625,-518.16,0.0,15547841,,U7706434


In [5]:
df1 = pm.get_positions_from_ib()
df1

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account
0,2024-01-28 22:15,EWT,STK,200.0,0.088016,44.256828,44.971699,0.016153,,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-01-28,8994.34,142.97,USD,0.0,U7706434
1,2024-01-28 22:15,IAU,STK,300.0,0.112115,38.454255,38.189701,-0.006880,,"Stock(conId=474829650, symbol='IAU', right='0'...",None,2024-01-28,11456.91,-79.37,USD,0.0,U7706434
2,2024-01-28 22:15,IAU,Call 39.0 20240216,-2.0,-0.000326,23.350200,0.166567,0.286659,,"Option(conId=671974250, symbol='IAU', lastTrad...",None,2024-01-28,-33.31,13.39,USD,0.0,U7706434
3,2024-01-28 22:15,INDA,STK,100.0,0.048117,49.712499,49.170799,-0.010897,,"Stock(conId=101484826, symbol='INDA', right='0...",None,2024-01-28,4917.08,-54.17,USD,0.0,U7706434
4,2024-01-28 22:15,IUSQ,STK,100.0,0.068911,65.602785,70.419998,0.073430,,"Stock(conId=239363241, symbol='IUSQ', right='0...",None,2024-01-28,7042.00,481.72,EUR,0.0,U7706434
5,2024-01-28 22:15,IWM,STK,100.0,0.191488,199.616920,195.679993,-0.019722,,"Stock(conId=9579970, symbol='IWM', right='0', ...",None,2024-01-28,19568.00,-393.69,USD,0.0,U7706434
6,2024-01-28 22:15,IWM,Call 199.0 20240216,-1.0,-0.002684,183.368900,2.743192,-0.495997,,"Option(conId=637114226, symbol='IWM', lastTrad...",None,2024-01-28,-274.32,-90.95,USD,0.0,U7706434
7,2024-01-28 22:15,PRX,STK,250.0,0.069136,26.561554,28.260000,0.063944,,"Stock(conId=382625193, symbol='PRX', right='0'...",None,2024-01-28,7065.00,424.61,EUR,0.0,U7706434
8,2024-01-28 22:15,SOXX,Put 525.0 20240216,-1.0,-0.000834,699.944800,0.852627,0.878187,,"Option(conId=653437229, symbol='SOXX', lastTra...",None,2024-01-28,-85.26,614.68,USD,0.0,U7706434
9,2024-01-28 22:15,TLT,STK,100.0,0.091859,99.051625,93.870003,-0.052312,,"Stock(conId=15547841, symbol='TLT', right='0',...",None,2024-01-28,9387.00,-518.16,USD,0.0,U7706434


In [6]:
df1.iloc[0].currency 
df1.iloc[0].marketValue

pm.convert_to_base_currency(df1.iloc[0]['% of nav'],df1.iloc[0].currency )


0.08114356161362417

Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: cashfarm; usfarm.nj; eufarm; usopt; usfarm; fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: cashfarm; usfarm.nj; eufarm; usopt; usfarm; euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trad

In [10]:

latest_positions = df1.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account
0,EWT,,STK,2024-01-28 22:15,200.0,0.088016,44.256828,44.971699,0.016153,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-01-28,8994.34,142.97,USD,0.0,U7706434
1,IAU,,Call 39.0 20240216,2024-01-28 22:15,-2.0,-0.000326,23.350200,0.166567,0.286659,"Option(conId=671974250, symbol='IAU', lastTrad...",None,2024-01-28,-33.31,13.39,USD,0.0,U7706434
2,IAU,,STK,2024-01-28 22:15,300.0,0.112115,38.454255,38.189701,-0.006880,"Stock(conId=474829650, symbol='IAU', right='0'...",None,2024-01-28,11456.91,-79.37,USD,0.0,U7706434
3,INDA,,STK,2024-01-28 22:15,100.0,0.048117,49.712499,49.170799,-0.010897,"Stock(conId=101484826, symbol='INDA', right='0...",None,2024-01-28,4917.08,-54.17,USD,0.0,U7706434
4,IUSQ,,STK,2024-01-28 22:15,100.0,0.068911,65.602785,70.419998,0.073430,"Stock(conId=239363241, symbol='IUSQ', right='0...",None,2024-01-28,7042.00,481.72,EUR,0.0,U7706434
5,IWM,,Call 199.0 20240216,2024-01-28 22:15,-1.0,-0.002684,183.368900,2.743192,-0.495997,"Option(conId=637114226, symbol='IWM', lastTrad...",None,2024-01-28,-274.32,-90.95,USD,0.0,U7706434
6,IWM,,STK,2024-01-28 22:15,100.0,0.191488,199.616920,195.679993,-0.019722,"Stock(conId=9579970, symbol='IWM', right='0', ...",None,2024-01-28,19568.00,-393.69,USD,0.0,U7706434
7,PRX,,STK,2024-01-28 22:15,250.0,0.069136,26.561554,28.260000,0.063944,"Stock(conId=382625193, symbol='PRX', right='0'...",None,2024-01-28,7065.00,424.61,EUR,0.0,U7706434
8,SOXX,,Put 525.0 20240216,2024-01-28 22:15,-1.0,-0.000834,699.944800,0.852627,0.878187,"Option(conId=653437229, symbol='SOXX', lastTra...",None,2024-01-28,-85.26,614.68,USD,0.0,U7706434
9,TLT,,Call 96.0 20240216,2024-01-28 22:15,-1.0,-0.000507,76.199700,0.517904,0.320334,"Option(conId=614854428, symbol='TLT', lastTrad...",None,2024-01-28,-51.79,24.41,USD,0.0,U7706434


# Add some data to portfolio

In [15]:
ac = initialize_db('db')
ac.list_libraries()
# general = ac.get_library('general')
portfolio_lib = ac.get_library('portfolio')
portfolio_lib.list_symbols()

['positions']

In [29]:
dummy_data = df.loc[0,:].copy()
dummy_data['position'] = dummy_data['position'] / 2
dummy_data['% of nav'] = dummy_data['% of nav'] / 2
dummy_data['averageCost'] = dummy_data['averageCost'] * 0.9
dummy_data['unrealizedPNL'] = (dummy_data['marketPrice'] - dummy_data['averageCost']) * dummy_data['position']
dummy_data['marketValue'] = dummy_data['marketPrice'] * dummy_data['position']
if dummy_data['asset class'] == 'STK' or dummy_data['asset class'] == 'FUT':
    pnl = ((dummy_data.marketPrice/(dummy_data.averageCost)) -1)
    pnl = pnl *(-1) if dummy_data.position < 0 else pnl
else:
    if dummy_data.position < 0:
        pnl = ((dummy_data.marketPrice/(dummy_data.averageCost/100)) -1) * (-1)
    else:
        pnl = ( (dummy_data.marketPrice/ (dummy_data.averageCost/100)) -1)
dummy_data['pnl %'] = pnl
dummy_data = pd.DataFrame(dummy_data).T


In [31]:
# Convert 'contract' column to string as ArcticDB may not handle custom objects well
dummy_data['contract'] = dummy_data['contract'].astype(str)

# Ensure that all other columns are of a consistent and compatible data type
# For example, convert all numeric columns to float
numeric_columns = ['position', '% of nav', 'averageCost', 'marketPrice', 'pnl %', 'marketValue', 'unrealizedPNL', 'realizedPNL']
for col in numeric_columns:
    dummy_data[col] = pd.to_numeric(dummy_data[col], errors='coerce')

# Now write the modified DataFrame to ArcticDB
portfolio_lib.write('positions', dummy_data)


VersionedItem(symbol='positions', library='portfolio', data=n/a, version=0, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [16]:
df_ac = portfolio_lib.read('positions').data
df_ac

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,marketValue,unrealizedPNL,realizedPNL,account
0,2024-01-21 15:27:35.463259,EWT,STK,100.0,0.04393,39.831146,44.210701,0.109953,,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.0701,437.955535,0.0,U7706434


#### Now, we will get ib positions and match them with arcticdb entries

In [12]:
df_ib = pm.get_positions_from_ib()

In [13]:
def match_ib_positions_with_arcticdb():
    library = ac.get_library('portfolio',create_if_missing=True)
    df_ac = library.read('positions').data
    latest_positions = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
    df_ib =  df_ib # self.get_positions_from_ib()

In [16]:
df_ac = portfolio_lib.read('positions').data
latest_positions_in_ac = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions_in_ac

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-21 15:27:35.463259,100.0,0.04393,39.831146,44.210701,0.109953,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.0701,437.955535,0.0,U7706434


In [17]:
def create_new_row():
    new_row = { 'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
                'symbol': symbol,'asset class': asset_class,
                'position':item.position,
                '% of nav':item.marketValue/total_equity,
                'averageCost': item.averageCost,
                'marketPrice': item.marketPrice,
                'pnl %': pnl,
                'strategy': '',
                'contract': str(item.contract),
                'marketValue': item.marketValue,
                'unrealizedPNL': item.unrealizedPNL,
                'realizedPNL': item.realizedPNL,
                'account': item.account}

def calculate_pnl(row):
    asset_class = row['asset_class']
    if asset_class == 'STK' or asset_class == 'FUT':
        pnl = ((row.marketPrice/(row.averageCost)) -1)
        pnl = pnl *(-1) if row.position < 0 else pnl
    else: # case for options only (be more specific)
        if row.position < 0:
            pnl = ((row.marketPrice/(row.averageCost/100)) -1) * (-1)
        else:
            pnl = ( (row.marketPrice/ (row.averageCost/100)) -1)
    return pnl



In [ ]:
for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']

    

In [18]:
df_merged = pd.DataFrame()
total_equity =  sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")

for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    # total_average_cost = row['averageCost']
    # total_market_price = row['marketPrice']

    strategy_entries_in_ac = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & 
                                           (latest_positions_in_ac['asset class'] == asset_class)]
    
    if not strategy_entries_in_ac.empty:
        strategy_entry = strategy_entries_in_ac.iloc[0]
        if total_position == strategy_entry.position:
            # Since the positions match, we use the latest data from ib and only add the strategy field
            row['strategy'] = strategy_entry.strategy
            df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)
        else:
            for entry in strategy_entries_in_ac:
                # update fields in: ['marketPrice', '% of nav','pnl %']
                strategy_entries_in_ac.loc[entry,'% of nav'] = ((strategy_entries_in_ac[entry,'position'] / total_position) * row['marketValue']) / total_equity
                # other update fields here

                # add entry to df_merged
            # ---------------------------------------------
                    
            # Calculate the averageCost, '% of nav','pnl %' and position for the residual
            assigned_position = strategy_entries_in_ac['position'].sum()
            residual_position = total_position - assigned_position

            # Calculate the average cost for the residual position
            ib_avg_cost = row['averageCost']
            ac_avg_cost = strategy_entries_in_ac.apply(lambda x: x['averageCost'] * (x['position'] / total_position), axis=1).sum()
            residual_avg_cost = (ib_avg_cost - ac_avg_cost) * (total_position / residual_position)
            #print(f"{symbol:} ib_avg_cost: {ib_avg_cost}, residual avg cost {residual_avg_cost}, ac_avg_cost {strategy_entries_in_ac.averageCost}")

            # do other calculations

            # concat residual position to df_merged
            
    else:
    # we have positions that are not in arcticdb that we should display & save as well
        pass
        

df_merged
    

<class 'pandas.core.series.Series'>


KeyError: ('symbol', 'position')

In [54]:
import pandas as pd

# Assuming df_ib is your DataFrame from ib.positions()
# Assuming latest_positions_in_ac is your DataFrame from ArcticDB with the latest position entries

df_merged = pd.DataFrame()
total_equity = sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")

# Iterate through the positions obtained from Interactive Brokers
for index, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    
    # Filter the ArcticDB DataFrame for entries with the same symbol and asset class
    strategy_entries_in_ac = latest_positions_in_ac[
        (latest_positions_in_ac['symbol'] == symbol) & 
        (latest_positions_in_ac['asset class'] == asset_class)
    ]
    
    if not strategy_entries_in_ac.empty:
        # Calculate the total position assigned to strategies in ArcticDB
        assigned_position = strategy_entries_in_ac['position'].sum()
        
        # If the total position from IB matches the sum of assigned positions in ArcticDB
        if total_position == assigned_position:
            # Use the row data and add the strategy info
            row['strategy'] = strategy_entries_in_ac['strategy'].values[0]  # Assuming one strategy per symbol/asset class
            df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)
        else:
            # There's a discrepancy between the positions, so we need to handle the residual
            # First, update the existing entries from ArcticDB
            strategy_entries_in_ac['marketPrice'] = row['marketPrice']
            strategy_entries_in_ac['% of nav'] = (strategy_entries_in_ac['position'] * row['marketPrice']) / total_equity
            strategy_entries_in_ac['pnl %'] = (strategy_entries_in_ac['marketPrice'] / strategy_entries_in_ac['averageCost'] - 1) * 100
            
            # Append the updated entries to df_merged
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # Now, calculate and append the residual position entry
            residual_position = total_position - assigned_position
            residual_avg_cost = ((row['averageCost'] * total_position) - (strategy_entries_in_ac['averageCost'] * assigned_position)) / residual_position
            residual_avg_cost = residual_avg_cost[0]
            residual_market_value = residual_position * row['marketPrice']
            residual_percent_nav = residual_market_value / total_equity
            residual_pnl_percent = ((row['marketPrice'] / residual_avg_cost) - 1) * 100
            
            residual_entry = row.copy()
            residual_entry['position'] = residual_position
            residual_entry['averageCost'] = residual_avg_cost
            residual_entry['marketValue'] = residual_market_value
            residual_entry['% of nav'] = residual_percent_nav
            residual_entry['pnl %'] = residual_pnl_percent
            residual_entry['strategy'] = ''  # No strategy assigned yet
            
            df_merged = pd.concat([df_merged, pd.DataFrame([residual_entry])], ignore_index=True)
            
    else:
        # If there are no entries in ArcticDB for this symbol/asset class, just append the row from ib
        df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

df_merged


,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-21 15:27:35.463259,100.0,0.044055,39.831146,45.020000,13.027129,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.070100,437.955535,0.0,U7706434
1,EWT,,STK,2024-01-26 21:23,100.0,0.044055,48.682511,45.020000,-7.523258,"Stock(conId=253190576, symbol='EWT', right='0'...",4502.000045,152.630000,0.0,U7706434
2,IAU,,STK,2024-01-26 21:23,300.0,0.112103,38.454255,38.183998,-0.007028,"Stock(conId=474829650, symbol='IAU', right='0'...",11455.200000,-81.080000,0.0,U7706434
3,IAU,,Call 39.0 20240216,2024-01-26 21:23,-2.0,-0.000324,23.350200,0.165493,0.291256,"Option(conId=671974250, symbol='IAU', lastTrad...",-33.100000,13.600000,0.0,U7706434
4,INDA,,STK,2024-01-26 21:23,100.0,0.048095,49.712499,49.146000,-0.011396,"Stock(conId=101484826, symbol='INDA', right='0...",4914.600000,-56.650000,0.0,U7706434
5,IUSQ,,STK,2024-01-26 21:23,100.0,0.068934,65.602785,70.440002,0.073735,"Stock(conId=239363241, symbol='IUSQ', right='0...",7044.000000,483.720000,0.0,U7706434
6,IWM,,STK,2024-01-26 21:23,100.0,0.191716,199.616920,195.903992,-0.018600,"Stock(conId=9579970, symbol='IWM', right='0', ...",19590.400000,-371.290000,0.0,U7706434
7,IWM,,Call 199.0 20240216,2024-01-26 21:23,-1.0,-0.002821,183.368900,2.883058,-0.572272,"Option(conId=637114226, symbol='IWM', lastTrad...",-288.310000,-104.940000,0.0,U7706434
8,PRX,,STK,2024-01-26 21:23,250.0,0.069140,26.561554,28.260000,0.063944,"Stock(conId=382625193, symbol='PRX', right='0'...",7065.000000,424.610000,0.0,U7706434
9,SOXX,,Put 525.0 20240216,2024-01-26 21:23,-1.0,-0.000827,699.944800,0.844985,0.879278,"Option(conId=653437229, symbol='SOXX', lastTra...",-84.500000,615.450000,0.0,U7706434


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde

In [48]:
latest_positions_in_ac.iloc[0]['contract']

AttributeError: 'str' object has no attribute 'to_dict'

In [33]:
import pandas as pd

df_merged = pd.DataFrame()  # Ensure df_merged is initialized as an empty DataFrame

# ... your code ...

for index, row in df_ib.iterrows():
    # ... your code ...
    df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

# ... your code ...


In [35]:
df_merged.append(row)

AttributeError: 'DataFrame' object has no attribute 'append'

In [22]:
# Assuming df_ib and latest_positions_in_ac are your DataFrames from IB and ArcticDB

for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    # total_average_cost = row['averageCost']
    # total_market_price = row['marketPrice']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & 
                                           (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        assigned_position = strategy_rows['position'].sum()
        residual_position = total_position - assigned_position

        if len(strategy_rows) == 1 and total_position == strategy_rows.iloc[0]['position']:
            # Case 1: Single strategy row and matching position
            # Update all columns in ArcticDB for this symbol & asset class combination except 'strategy'
            # Example: Update marketPrice and other related fields in ArcticDB
            pass

        else:
            # Case 2: Single/Multiple strategy rows and differing position
            # Calculate averageCost for the residual position
            assigned_average_cost = strategy_rows['averageCost'].iloc[0] if len(strategy_rows) == 1 else calculate_weighted_average_cost(strategy_rows)
            residual_average_cost = calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position)

            # Update marketPrice, marketValue, % of NAV, pnl%, unrealizedPNL for the assigned positions in ArcticDB
            # And calculate the corresponding values for the residual position
            pass

def calculate_weighted_average_cost(strategy_rows):
    # Implement logic to calculate a weighted average cost for multiple strategy rows
    pass

def calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position):
    # Implement logic to calculate the average cost for the residual position
    return ((total_average_cost * total_position) - (assigned_average_cost * assigned_position)) / (total_position - assigned_position)

# Further implement the functions or logic needed in 'pass' sections

1


In [57]:
for _, row in df_ib.iterrows():
    symbol = row.symbol
    asset_class = row['asset class']
    total_position = row['position']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        if len(strategy_rows) == 1 and total_position == strategy_rows.position:
            # update all cols in ac for this symbol & asset class combination except col strategies
            pass
        if len(strategy_rows) == 1 and total_position != strategy_rows.position:
            # 1. update cols marketPrice in ac from ib query and calculate new values for 'marketValue' (position*marketPrice), '% of nav', 'pnl%','unrealized'
            # calculate averageCost for the residual position (ib total averageCost = ac.averageCost * (ac.position/total_position) + residual.averageCost * (residual.position/total_position))
            # with the averageCost value and the position for the residual we can calculate 'marketValue', '% of nav', 'pnl%','unrealized'

            # if possible generalize the above for the cases where len(strategy_rows) > 1
            pass


1
EWT in ac
100.0 100.0


In [42]:
latest_positions['symbol'].values

array(['EWT'], dtype=object)